In [1]:
import gurobipy as gp

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
training_dataNSW = pd.read_csv('C:/Users/User/Desktop/DATA3001/JUPYTERDATAOUTPUT/Sarimax NSW.csv')
multi_NSW_OUTPUT = pd.read_csv('C:/Users/User/Desktop/DATA3001/JUPYTERDATAOUTPUT/multi.csv')
simple_NSW_OUTPUT= pd.read_csv('C:/Users/User/Desktop/DATA3001/JUPYTERDATAOUTPUT/simple.csv')


#training_dataQLD = pd.read_csv('C:/Users/User/Desktop/DATA3001/JUPYTERDATAOUTPUT/.csv')
#training_dataVIC = pd.read_csv('C:/Users/User/Desktop/DATA3001/JUPYTERDATAOUTPUT/.csv')
#training_dataSA = pd.read_csv('C:/Users/User/Desktop/DATA3001/JUPYTERDATAOUTPUT/.csv')
#training_dataTAS = pd.read_csv('C:/Users/User/Desktop/DATA3001/JUPYTERDATAOUTPUT/.csv')

In [4]:
#peaking window
training_dataNSW = training_dataNSW['RRP']
training_dataNSW

0      72.76286
1      74.35141
2      73.35515
3      69.65010
4      63.79023
         ...   
284    82.11968
285    76.68687
286    76.19823
287    73.92983
288    74.03793
Name: RRP, Length: 289, dtype: float64

In [5]:
#multi_NSW_OUTPUT['SETTLEMENTDATE']= pd.to_datetime(multi_NSW_OUTPUT['SETTLEMENTDATE'])
#training_dataQLD['SETTLEMENTDATE']= pd.to_datetime(training_dataQLD['SETTLEMENTDATE'])
#training_dataVIC['SETTLEMENTDATE']= pd.to_datetime(training_dataVIC['SETTLEMENTDATE'])
#training_dataSA['SETTLEMENTDATE']= pd.to_datetime(training_dataSA['SETTLEMENTDATE'])
#training_dataTAS['SETTLEMENTDATE']= pd.to_datetime(training_data['SETTLEMENTDATE'])


In [6]:
#multi_NSW_OUTPUT = multi_NSW_OUTPUT['pred']
#multi_NSW_OUTPUT[:24]['pred']

In [466]:
multi_NSW_OUTPUT = multi_NSW_OUTPUT.set_index('SETTLEMENTDATE')
#simple_NSW_OUTPUT 
#training_dataQLD = training_dataQLD.set_index('SETTLEMENTDATE')
#training_dataVIC = training_dataVIC.set_index('SETTLEMENTDATE')
#training_dataSA = training_dataSA.set_index('SETTLEMENTDATE')
#training_dataTAS = training_dataTAS.set_index('SETTLEMENTDATE')

In [509]:
#training_dataNSW.drop(['Unnamed: 0'], axis=1,inplace=True)
#training_data1MonthNSW = training_dataNSW['2008-07-01 00:00:00':'2008-08-01 00:00:00']
#testing_data1dayNSW = training_dataNSW['2016-07-02 00:00:05':'2016-10-02 00:00:05']
#training_dataNSW['2008-07-01 00:00:00':'2008-08-01 00:00:00']

simple_NSW_OUTPUT = simple_NSW_OUTPUT['pred']


In [477]:
training_dataNSWsarimax = training_dataNSW['2018-06-30 00:00:00':'2018-07-01 00:00:00']
#training_dataNSWARIMAx = training_dataNSW['2016-07-02 00:00:05':'2016-10-03 00:00:05']
#training_dataNSWLR = #training_dataNSW['2016-07-02 00:00:05':'2016-10-03 00:00:05']

#training_dataVICsarimax= #training_dataVIC['2008-08-01 00:00:00':'2008-08-02 00:00:00']
#training_dataVICARIMAx= training_dataVIC['2008-08-01 00:00:00':'2008-08-02 00:00:00']
#training_dataVICLR= #training_dataVIC['2008-08-01 00:00:00':'2008-08-02 00:00:00']

#training_dataQLDsarimax = #training_dataQLD['2008-08-01 00:00:00':'2008-08-02 00:00:00']
#training_dataQLDARIMAx= training_dataQLD['2008-08-01 00:00:00':'2008-08-02 00:00:00']
#training_dataQLDLR= #training_dataQLD['2008-08-01 00:00:00':'2008-08-02 00:00:00']#

#training_dataSAsarimax= #training_dataSA['2008-08-01 00:00:00':'2008-08-02 00:00:00']
#training_dataSAARIMAx= training_dataSA['2008-08-01 00:00:00':'2008-08-02 00:00:00']
#training_dataSALR= #training_dataSA['2008-08-01 00:00:00':'2008-08-02 00:00:00']

#training_dataTASsarimax= #training_dataTAS['2008-08-01 00:00:00':'2008-08-02 00:00:00']
#training_dataTASARIMAx= training_dataTAS['2008-08-01 00:00:00':'2008-08-02 00:00:00']
#training_dataTASLR= #training_dataTAS['2008-08-01 00:00:00':'2008-08-02 00:00:00']

In [605]:
#list(simple_NSW_OUTPUT['pred'])

In [5]:
#n would be the (frequency_of_decision_making_for_battery_in_minutes)/5
#lst is the price that is predicted for the upcoming day. So, there should be 288 observations in lst. 
def list_divided_averages(lst, n):
    list_divided = [lst[i:i + n] for i in range(0, len(lst), n)]
    list_of_avrgs = []
    for lisst in list_divided:
        list_of_avrgs.append(round(sum(lisst) / len(lisst), 5))
    return list_of_avrgs

In [6]:
#price is the predicted prices (list format) for the upcoming 24 hours from our models. Hence, there should be 288 observations. 
#price = testing_dateTHAZSARIMAX_RES

#the 60 in the second parameter below represents the frequency of update.
def cost_coefficient_generator(price_predictions, frequency_of_update_in_min):
    n = int(frequency_of_update_in_min/5)
    cost_coefficients = list_divided_averages(list(training_dataNSW), n)
    return cost_coefficients

### If we decide to update our model every 1 hour then, we are making an assumption that the average of the prices of the 12 intervals of 5 minutes that went past is a good indicator of what the price was for that hour. 

In [49]:
frequencyOUpdate = 5 #(in minutes) - For e.g frequencyOUpdate=60 for deciding every one hour if we want to charge or discharge.

cost_coefficient_column=cost_coefficient_generator(list(training_dataNSW), frequencyOUpdate)

In [50]:
len(cost_coefficient_column)

289

## Optimization model

In [67]:
%reset_selective model3

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [68]:
#model3.reset()

In [69]:
model3 = gp.Model()

DischargeBattery2 = {}
for h in range(0,len(cost_coefficient_column)):
    DischargeBattery2[h] = model3.addVar(lb=0, ub=1,obj=cost_coefficient_column[h]*(1/60)*(frequencyOUpdate), vtype=GRB.CONTINUOUS)

ChargeBattery2 = {}
DispatchGen = {}
for h in range(0,len(cost_coefficient_column)):
    ChargeBattery2[h] = model3.addVar(lb=-1, ub=0,obj=1*cost_coefficient_column[h]*(1/60)*(frequencyOUpdate), vtype=GRB.CONTINUOUS)
    DispatchGen[h] = model3.addVar(lb=0, ub=2, vtype = GRB.CONTINUOUS)

#EnergyInStorage = {}
#for m in range(0, len(cost_coefficient_column)+1):
#    EnergyInStorage[h] = model3.addVar(lb=0, ub=2, vtype = GRB.CONTINUOUS)
EnergyInStorage = [model3.addVar(lb=0, ub=2, vtype=GRB.CONTINUOUS) for h in range(0, len(cost_coefficient_column)+1)]

model3.modelSense=GRB.MAXIMIZE
model3.update()

In [70]:

for h in range(0, len(cost_coefficient_column)-1):
    model3.addConstr(EnergyInStorage[h] >= DischargeBattery2[h]*(frequencyOUpdate/60))
    EnergyInStorage = [model3.addVar(lb=0, ub=2, vtype=GRB.CONTINUOUS)  for h in range(0, len(cost_coefficient_column)+1)]

#model3.addConstr(EnergyInStorage[0] == 0)
for h in range(0,len(cost_coefficient_column)):
    model3.addConstr(EnergyInStorage[h+1] == EnergyInStorage[h] - 0.81*ChargeBattery2[h]*(frequencyOUpdate/60) - DischargeBattery2[h]*(frequencyOUpdate/60))


for h in range(0, len(cost_coefficient_column)):
    model3.addConstr(EnergyInStorage[h] <= 2)

for h in range(0, len(cost_coefficient_column)):
    model3.addConstr(DispatchGen[h] == DischargeBattery2[h] + ChargeBattery2[h])
        

In [71]:
model3.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1155 rows, 84677 columns and 2888 nonzeros
Model fingerprint: 0xb23f7120
Coefficient statistics:
  Matrix range     [7e-02, 1e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 577 rows and 83809 columns
Presolve time: 0.07s
Presolved: 578 rows, 868 columns, 1734 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6782057e+03   2.915083e+03   0.000000e+00      0s
     199    2.0508651e+02   0.000000e+00   0.000000e+00      0s

Solved in 199 iterations and 0.13 seconds
Optimal objective  2.050865100e+02


In [72]:
[ DischargeBattery2[i].x  for i in range(0, 24)]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [73]:
[ChargeBattery2[i].x  for i in range(0, 24)]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]